In [1]:
#Ahmet Şeref Eker
#2210356098 75
#Dear Hocam, if you are going to run this code, please consider the pandas encoding in line 75(in scale function) because i am writing this code on macOS

import pandas as pd
import numpy as np

def decode(personality):  # this function is for decoding personalities for the information part at the end
	if personality == 0:
		return "ESTJ"
	elif personality == 1:
		return "ENTJ"
	elif personality == 2:
		return "ESFJ"
	elif personality == 3:
		return "ENFJ"
	elif personality == 4:
		return "ISTJ"
	elif personality == 5:
		return "ISFJ"
	elif personality == 6:
		return "INTJ"
	elif personality == 7:
		return "INFJ"
	elif personality == 8:
		return "ESTP"
	elif personality == 9:
		return "ESFP"
	elif personality == 10:
		return "ENTP"
	elif personality == 11:
		return "ENFP"
	elif personality == 12:
		return "ISTP"
	elif personality == 13:
		return "ISFP"
	elif personality == 14:
		return "INTP"
	elif personality == 15:
		return "INFP"

def encode(personality):  # this function is for encoding the personalities to integer
	if personality == "ESTJ":
		return 0
	elif personality == "ENTJ":
		return 1
	elif personality == "ESFJ":
		return 2
	elif personality == "ENFJ":
		return 3
	elif personality == "ISTJ":
		return 4
	elif personality == "ISFJ":
		return 5
	elif personality == "INTJ":
		return 6
	elif personality == "INFJ":
		return 7
	elif personality == "ESTP":
		return 8
	elif personality == "ESFP":
		return 9
	elif personality == "ENTP":
		return 10
	elif personality == "ENFP":
		return 11
	elif personality == "ISTP":
		return 12
	elif personality == "ISFP":
		return 13
	elif personality == "INTP":
		return 14
	elif personality == "INFP":
		return 15

def scale():  # scaling dataframe and returning a data array and a personality array

	#region           reads the csv file as df, deletes Response Id and personality, turns to array, encodes personality -> ------df_arr------  ------personality_arr------
	df_raw = pd.read_csv("16P.csv", encoding='mac_roman')
	personality_list = list(df_raw["Personality"])
	df = df_raw.drop(columns=["Response Id","Personality"])
	df_arr = np.array(df)
	for t in range(len(personality_list)):
		personality_list[t] = encode(personality_list[t])
	personality_arr = np.array(personality_list)
	#endregion
	
	return df_arr,personality_arr

def determine_max_and_min(data_array):  # [[1,2,3,4] get array as inputs an returns returns min and max -> ------max,min--------
										#  [2,3,4,5]
	max = data_array[0,0]				#  [3,4,5,6]
	min = data_array[0,0]               #  [4,5,6,7]
	for row in data_array:              #  [5,6,7,8]
		for member in row:              #  [6,7,8,9]]
			if max > member:            
				max = member
			if min < member:
				min = member
	return max,min                      # returns 1,9   min,max

def feature_normalization(data_array):  # normalizes every value of given array, returns normalized array, reassignment needed afterwards
	data_array = np.asfarray(data_array)
	min , max = determine_max_and_min(data_array)
	for i,row in enumerate(data_array):
		for t,member in enumerate(row):
			data_array[i,t] = (member - min)/(max - min)
	return data_array

def slicing_train_and_test(data_array,test_index,personality_arr):  # returns test(k) and train(4k) part in this order
	length = len(data_array)
	part_len = (length//5)
	if test_index ==1:
		personality_test_arr = personality_arr[:part_len]
		personality_train_arr = personality_arr[part_len:]
		return data_array[:part_len], data_array[part_len:], personality_test_arr, personality_train_arr
	elif test_index ==2:
		personality_test_arr = personality_arr[part_len:part_len*2]
		personality_train_arr = np.concatenate((personality_arr[:part_len], personality_arr[part_len*2:]))
		return data_array[part_len:part_len*2], np.vstack((data_array[:part_len],data_array[part_len*2:])), personality_test_arr, personality_train_arr
	elif test_index ==3:
		personality_test_arr = personality_arr[part_len*2:part_len*3]
		personality_train_arr = np.concatenate((personality_arr[:part_len*2], personality_arr[part_len*3:]))
		return data_array[part_len*2:part_len*3], np.vstack((data_array[:part_len*2],data_array[part_len*3:])), personality_test_arr, personality_train_arr
	elif test_index ==4:
		personality_test_arr = personality_arr[part_len*3:part_len*4]
		personality_train_arr = np.concatenate((personality_arr[:part_len*3], personality_arr[part_len*4:]))
		return data_array[part_len*3:part_len*4], np.vstack((data_array[:part_len*3],data_array[part_len*4:])), personality_test_arr, personality_train_arr
	elif test_index ==5:
		personality_test_arr = personality_arr[part_len*4:]
		personality_train_arr = personality_arr[:part_len*4]
		return data_array[part_len*4:], data_array[:part_len*4], personality_test_arr, personality_train_arr

def calculate_accuracy(conf_matrix):  # returns accuracy
	true = sum([conf_matrix[i,i] for i in range(len(conf_matrix))])
	all = np.sum((conf_matrix))
	return true/all

def knn(slice_point,k,is_feature_normalization): # return confussion_matrix and misclassified_samples
	if slice_point==5:  # determining parts length for matrix multiplication
		test_len , train_len = 12003 , 47996
	else:
		test_len , train_len = 11999 , 48000
	df_arr , personality_arr = scale()
	if is_feature_normalization==True:
		df_arr = feature_normalization(df_arr)
	test_arr, train_arr, per_test_arr, per_train_arr = slicing_train_and_test(df_arr,slice_point,personality_arr)
	confussion_matrix = np.zeros((16,16))  # creating a confussion matrix
	test_part = (test_arr*test_arr).sum(axis=1)
	test_part = test_part.reshape((test_len,1))*np.ones(shape=(1,train_len))
	train_part = (train_arr*train_arr).sum(axis=1)
	train_part = train_part*np.ones(shape=(test_len,1))
	sum =  test_part + train_part -2*test_arr.dot(train_arr.T)  #  matrix multiplication
	nearest_personalities = per_train_arr[np.argsort(sum)[:,:k]]  #  nearest points indexs
	misclassified_points_indexs = [person for person in range(len(nearest_personalities)) if list(np.bincount(nearest_personalities[person])).count(np.max(np.bincount(nearest_personalities[person]))) > 1]  # misclassified samples
	predict = np.array([np.bincount(row).argmax() for row in nearest_personalities])  # predict value
	for m in range(len(per_test_arr)):
		confussion_matrix[per_test_arr[m],predict[m]] +=1  # adding informations to confussing matrix
	def calculate_precision_recall_and_print(conf_matrix):  # calculating and printing the precision and recall
		def precision_recall_calculate(per):  # returns precision and recall values
			tp = conf_matrix[per, per]  # true positive
			fn = np.sum(np.concatenate((conf_matrix[per,:per],conf_matrix[per,per+1:])))  # false negative
			fp = np.sum(np.concatenate((conf_matrix[:per,per],conf_matrix[per+1:,per])))  # false positive
			#tn = np.sum(conf_matrix)-(tp+fn+fp)  true negatives
			def precision():
				return tp/(tp+fp)
			def recall():
				return tp/(tp+fn)
			return precision(),recall()
		def print_calculations():
			sum_precision = 0
			sum_recall = 0
			for row in range(len(conf_matrix)):
				precision , recall = precision_recall_calculate(row)
				sum_precision+=precision
				sum_recall+=recall
				personality = decode(row)
				print(personality)
				print("Precision =", precision)
				print("Recall =", recall,end="\n\n")
			print("Acuracy = {}".format(calculate_accuracy(conf_matrix)))
			print("Average Precision = {}".format(sum_precision/16))
			print("Average Recall = {}\n".format(sum_recall/16))
			print("Total Misclassified Samples :", len(misclassified_points_indexs))
			if len(misclassified_points_indexs) != 0:
				print("Misclassified Samples Index :", misclassified_points_indexs)
		print_calculations()
	calculate_precision_recall_and_print(confussion_matrix)

#  example run  --->     knn(3, 7, False)    -->    k-fold  ,  k nearest  ,  feature normalization

In [2]:
knn(1,1,False)  #   1.fold  ,  k = 1  ,  without feature normalization

ESTJ
Precision = 0.973718791064389
Recall = 0.9775725593667546

ENTJ
Precision = 0.9809885931558935
Recall = 0.979746835443038

ESFJ
Precision = 0.9826435246995995
Recall = 0.9879194630872483

ENFJ
Precision = 0.9689608636977058
Recall = 0.9862637362637363

ISTJ
Precision = 0.9755102040816327
Recall = 0.9768392370572208

ISFJ
Precision = 0.9756410256410256
Recall = 0.9819354838709677

INTJ
Precision = 0.9907038512616202
Recall = 0.9777195281782438

INFJ
Precision = 0.9731457800511509
Recall = 0.9870298313878081

ESTP
Precision = 0.982620320855615
Recall = 0.9839357429718876

ESFP
Precision = 0.9768392370572208
Recall = 0.9741847826086957

ENTP
Precision = 0.9878542510121457
Recall = 0.973404255319149

ENFP
Precision = 0.9847009735744089
Recall = 0.9725274725274725

ISTP
Precision = 0.9769335142469471
Recall = 0.9795918367346939

ISFP
Precision = 0.9830287206266318
Recall = 0.9817470664928292

INTP
Precision = 0.9781718963165075
Recall = 0.9728629579375848

INFP
Precision = 0.9712722298

In [3]:
knn(1,3,False)  #   1.fold  ,  k = 3  ,  without feature normalization

ESTJ
Precision = 0.9880478087649402
Recall = 0.9815303430079155

ENTJ
Precision = 0.9824561403508771
Recall = 0.9924050632911392

ESFJ
Precision = 0.9801324503311258
Recall = 0.9932885906040269

ENFJ
Precision = 0.9850543478260869
Recall = 0.9958791208791209

ISTJ
Precision = 0.9864130434782609
Recall = 0.989100817438692

ISFJ
Precision = 0.9846153846153847
Recall = 0.9909677419354839

INTJ
Precision = 0.996031746031746
Recall = 0.9868938401048493

INFJ
Precision = 0.982051282051282
Recall = 0.993514915693904

ESTP
Precision = 0.9893048128342246
Recall = 0.9906291834002677

ESFP
Precision = 0.9876543209876543
Recall = 0.9782608695652174

ENTP
Precision = 0.9973154362416108
Recall = 0.988031914893617

ENFP
Precision = 0.9930747922437673
Recall = 0.9848901098901099

ISTP
Precision = 0.9877049180327869
Recall = 0.9836734693877551

ISFP
Precision = 0.9909208819714657
Recall = 0.9960886571056062

INTP
Precision = 0.9918032786885246
Recall = 0.9850746268656716

INFP
Precision = 0.99449035812

In [4]:
knn(1,5,False)  #   1.fold  ,  k = 5  ,  without feature normalization

ESTJ
Precision = 0.9880636604774535
Recall = 0.9828496042216359

ENTJ
Precision = 0.9861635220125786
Recall = 0.9924050632911392

ESFJ
Precision = 0.9801324503311258
Recall = 0.9932885906040269

ENFJ
Precision = 0.9890859481582538
Recall = 0.9958791208791209

ISTJ
Precision = 0.9877551020408163
Recall = 0.989100817438692

ISFJ
Precision = 0.9871630295250321
Recall = 0.9922580645161291

INTJ
Precision = 0.9934123847167325
Recall = 0.9882044560943644

INFJ
Precision = 0.9833119383825417
Recall = 0.993514915693904

ESTP
Precision = 0.9892904953145917
Recall = 0.9892904953145917

ESFP
Precision = 0.9890260631001372
Recall = 0.9796195652173914

ENTP
Precision = 0.9973154362416108
Recall = 0.988031914893617

ENFP
Precision = 0.9930939226519337
Recall = 0.9876373626373627

ISTP
Precision = 0.9877384196185286
Recall = 0.9863945578231292

ISFP
Precision = 0.9934980494148244
Recall = 0.9960886571056062

INTP
Precision = 0.9918256130790191
Recall = 0.9877883310719131

INFP
Precision = 0.993131868

In [5]:
knn(1,7,False)  #   1.fold  ,  k = 7  ,  without feature normalization

ESTJ
Precision = 0.9880636604774535
Recall = 0.9828496042216359

ENTJ
Precision = 0.9861635220125786
Recall = 0.9924050632911392

ESFJ
Precision = 0.9801324503311258
Recall = 0.9932885906040269

ENFJ
Precision = 0.9890859481582538
Recall = 0.9958791208791209

ISTJ
Precision = 0.9877551020408163
Recall = 0.989100817438692

ISFJ
Precision = 0.9884318766066839
Recall = 0.9922580645161291

INTJ
Precision = 0.9934123847167325
Recall = 0.9882044560943644

INFJ
Precision = 0.9845758354755784
Recall = 0.993514915693904

ESTP
Precision = 0.9906417112299465
Recall = 0.9919678714859438

ESFP
Precision = 0.987688098495212
Recall = 0.9809782608695652

ENTP
Precision = 0.9973154362416108
Recall = 0.988031914893617

ENFP
Precision = 0.9930843706777317
Recall = 0.9862637362637363

ISTP
Precision = 0.9877384196185286
Recall = 0.9863945578231292

ISFP
Precision = 0.9934980494148244
Recall = 0.9960886571056062

INTP
Precision = 0.9918256130790191
Recall = 0.9877883310719131

INFP
Precision = 0.9945054945

In [6]:
knn(1,9,False)  #   1.fold  ,  k = 9  ,  without feature normalization

ESTJ
Precision = 0.9880636604774535
Recall = 0.9828496042216359

ENTJ
Precision = 0.9861635220125786
Recall = 0.9924050632911392

ESFJ
Precision = 0.9801324503311258
Recall = 0.9932885906040269

ENFJ
Precision = 0.9890859481582538
Recall = 0.9958791208791209

ISTJ
Precision = 0.9877384196185286
Recall = 0.9877384196185286

ISFJ
Precision = 0.9884318766066839
Recall = 0.9922580645161291

INTJ
Precision = 0.9960369881109643
Recall = 0.9882044560943644

INFJ
Precision = 0.9845758354755784
Recall = 0.993514915693904

ESTP
Precision = 0.9906291834002677
Recall = 0.9906291834002677

ESFP
Precision = 0.989041095890411
Recall = 0.9809782608695652

ENTP
Precision = 0.9973154362416108
Recall = 0.988031914893617

ENFP
Precision = 0.9930939226519337
Recall = 0.9876373626373627

ISTP
Precision = 0.9877551020408163
Recall = 0.9877551020408163

ISFP
Precision = 0.9934980494148244
Recall = 0.9960886571056062

INTP
Precision = 0.989145183175034
Recall = 0.989145183175034

INFP
Precision = 0.99313186813

In [7]:
knn(2,1,False)  #   2.fold  ,  k = 1  ,  without feature normalization

ESTJ
Precision = 0.9823455233291298
Recall = 0.9811083123425692

ENTJ
Precision = 0.9713168187744459
Recall = 0.9815546772068511

ESFJ
Precision = 0.9746666666666667
Recall = 0.9759679572763685

ENFJ
Precision = 0.9701298701298702
Recall = 0.9816031537450722

ISTJ
Precision = 0.9739726027397261
Recall = 0.9779917469050894

ISFJ
Precision = 0.9791666666666666
Recall = 0.9835575485799701

INTJ
Precision = 0.9649122807017544
Recall = 0.9821428571428571

INFJ
Precision = 0.9893475366178429
Recall = 0.9815059445178336

ESTP
Precision = 0.9815059445178336
Recall = 0.9854111405835544

ESFP
Precision = 0.9859872611464968
Recall = 0.9772727272727273

ENTP
Precision = 0.9635627530364372
Recall = 0.9780821917808219

ENFP
Precision = 0.9775725593667546
Recall = 0.9814569536423841

ISTP
Precision = 0.9755434782608695
Recall = 0.9742198100407056

ISFP
Precision = 0.9850136239782016
Recall = 0.9601593625498008

INTP
Precision = 0.9769647696476965
Recall = 0.9575033200531209

INFP
Precision = 0.974965

In [8]:
knn(2,3,False)  #   2.fold  ,  k = 3  ,  without feature normalization

ESTJ
Precision = 0.9886792452830189
Recall = 0.9899244332493703

ENTJ
Precision = 0.9933862433862434
Recall = 0.9894598155467721

ESFJ
Precision = 0.9827814569536424
Recall = 0.9906542056074766

ENFJ
Precision = 0.989501312335958
Recall = 0.9908015768725361

ISTJ
Precision = 0.9876373626373627
Recall = 0.9889958734525447

ISFJ
Precision = 0.9880597014925373
Recall = 0.9895366218236173

INTJ
Precision = 0.9823232323232324
Recall = 0.9923469387755102

INFJ
Precision = 0.9947019867549669
Recall = 0.9920739762219286

ESTP
Precision = 0.9907161803713528
Recall = 0.9907161803713528

ESFP
Precision = 0.9961783439490446
Recall = 0.9873737373737373

ENTP
Precision = 0.984993178717599
Recall = 0.989041095890411

ENFP
Precision = 0.9893758300132802
Recall = 0.9867549668874173

ISTP
Precision = 0.9904632152588556
Recall = 0.9864314789687924

ISFP
Precision = 0.9906166219839142
Recall = 0.9814077025232404

INTP
Precision = 0.9853528628495339
Recall = 0.9827357237715804

INFP
Precision = 0.986301369

In [9]:
knn(2,5,False)  #   2.fold  ,  k = 5  ,  without feature normalization

ESTJ
Precision = 0.9899244332493703
Recall = 0.9899244332493703

ENTJ
Precision = 0.9947019867549669
Recall = 0.9894598155467721

ESFJ
Precision = 0.9828042328042328
Recall = 0.9919893190921228

ENFJ
Precision = 0.989501312335958
Recall = 0.9908015768725361

ISTJ
Precision = 0.990358126721763
Recall = 0.9889958734525447

ISFJ
Precision = 0.9880418535127056
Recall = 0.9880418535127056

INTJ
Precision = 0.9835858585858586
Recall = 0.9936224489795918

INFJ
Precision = 0.9947019867549669
Recall = 0.9920739762219286

ESTP
Precision = 0.9894179894179894
Recall = 0.9920424403183024

ESFP
Precision = 0.9961783439490446
Recall = 0.9873737373737373

ENTP
Precision = 0.9836734693877551
Recall = 0.9904109589041096

ENFP
Precision = 0.9893758300132802
Recall = 0.9867549668874173

ISTP
Precision = 0.9917920656634747
Recall = 0.9837177747625508

ISFP
Precision = 0.9906417112299465
Recall = 0.9840637450199203

INTP
Precision = 0.9853528628495339
Recall = 0.9827357237715804

INFP
Precision = 0.98632010

In [10]:
knn(2,7,False)  #   2.fold  ,  k = 7  ,  without feature normalization

ESTJ
Precision = 0.9911727616645649
Recall = 0.9899244332493703

ENTJ
Precision = 0.9947019867549669
Recall = 0.9894598155467721

ESFJ
Precision = 0.9815059445178336
Recall = 0.9919893190921228

ENFJ
Precision = 0.9895150720838795
Recall = 0.9921156373193167

ISTJ
Precision = 0.990358126721763
Recall = 0.9889958734525447

ISFJ
Precision = 0.9880597014925373
Recall = 0.9895366218236173

INTJ
Precision = 0.9836065573770492
Recall = 0.9948979591836735

INFJ
Precision = 0.9947019867549669
Recall = 0.9920739762219286

ESTP
Precision = 0.9907038512616202
Recall = 0.9893899204244032

ESFP
Precision = 0.9961783439490446
Recall = 0.9873737373737373

ENTP
Precision = 0.9850136239782016
Recall = 0.9904109589041096

ENFP
Precision = 0.9893758300132802
Recall = 0.9867549668874173

ISTP
Precision = 0.9918032786885246
Recall = 0.9850746268656716

ISFP
Precision = 0.9906417112299465
Recall = 0.9840637450199203

INTP
Precision = 0.9853528628495339
Recall = 0.9827357237715804

INFP
Precision = 0.9863201

In [11]:
knn(2,9,False)  #   2.fold  ,  k = 9  ,  without feature normalization

ESTJ
Precision = 0.9911727616645649
Recall = 0.9899244332493703

ENTJ
Precision = 0.9947019867549669
Recall = 0.9894598155467721

ESFJ
Precision = 0.9841059602649007
Recall = 0.9919893190921228

ENFJ
Precision = 0.9895150720838795
Recall = 0.9921156373193167

ISTJ
Precision = 0.990358126721763
Recall = 0.9889958734525447

ISFJ
Precision = 0.9880597014925373
Recall = 0.9895366218236173

INTJ
Precision = 0.9848293299620733
Recall = 0.9936224489795918

INFJ
Precision = 0.9947019867549669
Recall = 0.9920739762219286

ESTP
Precision = 0.9894179894179894
Recall = 0.9920424403183024

ESFP
Precision = 0.9961783439490446
Recall = 0.9873737373737373

ENTP
Precision = 0.9850136239782016
Recall = 0.9904109589041096

ENFP
Precision = 0.9893758300132802
Recall = 0.9867549668874173

ISTP
Precision = 0.9918256130790191
Recall = 0.9877883310719131

ISFP
Precision = 0.9906417112299465
Recall = 0.9840637450199203

INTP
Precision = 0.9853528628495339
Recall = 0.9827357237715804

INFP
Precision = 0.9876712

In [12]:
knn(3,1,False)  #   3.fold  ,  k = 1  ,  without feature normalization

ESTJ
Precision = 0.9833101529902643
Recall = 0.9833101529902643

ENTJ
Precision = 0.9682758620689655
Recall = 0.9831932773109243

ESFJ
Precision = 0.9836734693877551
Recall = 0.9743935309973046

ENFJ
Precision = 0.9793388429752066
Recall = 0.9820441988950276

ISTJ
Precision = 0.9777777777777777
Recall = 0.9805013927576601

ISFJ
Precision = 0.9702233250620348
Recall = 0.9799498746867168

INTJ
Precision = 0.968
Recall = 0.9784366576819407

INFJ
Precision = 0.9917241379310345
Recall = 0.9663978494623656

ESTP
Precision = 0.9802890932982917
Recall = 0.9751633986928104

ESFP
Precision = 0.9777195281782438
Recall = 0.9738903394255874

ENTP
Precision = 0.9752925877763329
Recall = 0.9778357235984355

ENFP
Precision = 0.9777777777777777
Recall = 0.9842105263157894

ISTP
Precision = 0.9801734820322181
Recall = 0.9789603960396039

ISFP
Precision = 0.9803646563814866
Recall = 0.9776223776223776

INTP
Precision = 0.9778357235984355
Recall = 0.9664948453608248

INFP
Precision = 0.9585561497326203
Re

In [13]:
knn(3,3,False)  #   3.fold  ,  k = 3  ,  without feature normalization

ESTJ
Precision = 0.9944289693593314
Recall = 0.9930458970792768

ENTJ
Precision = 0.9820193637621023
Recall = 0.9943977591036415

ESFJ
Precision = 0.9905277401894452
Recall = 0.9865229110512129

ENFJ
Precision = 0.9876373626373627
Recall = 0.9930939226519337

ISTJ
Precision = 0.9874301675977654
Recall = 0.9846796657381616

ISFJ
Precision = 0.9801242236024845
Recall = 0.9887218045112782

INTJ
Precision = 0.982620320855615
Recall = 0.9905660377358491

INFJ
Precision = 0.9986376021798365
Recall = 0.9852150537634409

ESTP
Precision = 0.9895561357702349
Recall = 0.9908496732026144

ESFP
Precision = 0.9908015768725361
Recall = 0.9843342036553525

ENTP
Precision = 0.9921465968586387
Recall = 0.9882659713168188

ENFP
Precision = 0.9947019867549669
Recall = 0.9881578947368421

ISTP
Precision = 0.9925093632958801
Recall = 0.9839108910891089

ISFP
Precision = 0.9901547116736991
Recall = 0.9846153846153847

INTP
Precision = 0.985897435897436
Recall = 0.990979381443299

INFP
Precision = 0.981333333

In [14]:
knn(3,5,False)  #   3.fold  ,  k = 5  ,  without feature normalization

ESTJ
Precision = 0.9944367176634215
Recall = 0.9944367176634215

ENTJ
Precision = 0.9861111111111112
Recall = 0.9943977591036415

ESFJ
Precision = 0.9905405405405405
Recall = 0.9878706199460916

ENFJ
Precision = 0.9876373626373627
Recall = 0.9930939226519337

ISTJ
Precision = 0.9888268156424581
Recall = 0.9860724233983287

ISFJ
Precision = 0.9838107098381071
Recall = 0.9899749373433584

INTJ
Precision = 0.9813333333333333
Recall = 0.9919137466307277

INFJ
Precision = 0.998639455782313
Recall = 0.9865591397849462

ESTP
Precision = 0.9908496732026144
Recall = 0.9908496732026144

ESFP
Precision = 0.9921156373193167
Recall = 0.9856396866840731

ENTP
Precision = 0.9908496732026144
Recall = 0.9882659713168188

ENFP
Precision = 0.9947019867549669
Recall = 0.9881578947368421

ISTP
Precision = 0.9925187032418953
Recall = 0.9851485148514851

ISFP
Precision = 0.9915492957746479
Recall = 0.9846153846153847

INTP
Precision = 0.9871630295250321
Recall = 0.990979381443299

INFP
Precision = 0.98002663

In [15]:
knn(3,7,False)  #   3.fold  ,  k = 7  ,  without feature normalization

ESTJ
Precision = 0.9944367176634215
Recall = 0.9944367176634215

ENTJ
Precision = 0.9874826147426982
Recall = 0.9943977591036415

ESFJ
Precision = 0.99055330634278
Recall = 0.9892183288409704

ENFJ
Precision = 0.9876373626373627
Recall = 0.9930939226519337

ISTJ
Precision = 0.9888579387186629
Recall = 0.9888579387186629

ISFJ
Precision = 0.9838107098381071
Recall = 0.9899749373433584

INTJ
Precision = 0.9813581890812251
Recall = 0.9932614555256065

INFJ
Precision = 0.998639455782313
Recall = 0.9865591397849462

ESTP
Precision = 0.9908496732026144
Recall = 0.9908496732026144

ESFP
Precision = 0.9921156373193167
Recall = 0.9856396866840731

ENTP
Precision = 0.9921465968586387
Recall = 0.9882659713168188

ENFP
Precision = 0.9947019867549669
Recall = 0.9881578947368421

ISTP
Precision = 0.9937578027465668
Recall = 0.9851485148514851

ISFP
Precision = 0.9915492957746479
Recall = 0.9846153846153847

INTP
Precision = 0.9871630295250321
Recall = 0.990979381443299

INFP
Precision = 0.9813333333

In [16]:
knn(3,9,False)  #   3.fold  ,  k = 9  ,  without feature normalization

ESTJ
Precision = 0.9944444444444445
Recall = 0.9958275382475661

ENTJ
Precision = 0.9874826147426982
Recall = 0.9943977591036415

ESFJ
Precision = 0.99055330634278
Recall = 0.9892183288409704

ENFJ
Precision = 0.9889958734525447
Recall = 0.9930939226519337

ISTJ
Precision = 0.9888579387186629
Recall = 0.9888579387186629

ISFJ
Precision = 0.9838107098381071
Recall = 0.9899749373433584

INTJ
Precision = 0.9813581890812251
Recall = 0.9932614555256065

INFJ
Precision = 0.998639455782313
Recall = 0.9865591397849462

ESTP
Precision = 0.9908376963350786
Recall = 0.9895424836601308

ESFP
Precision = 0.9908136482939632
Recall = 0.9856396866840731

ENTP
Precision = 0.9921465968586387
Recall = 0.9882659713168188

ENFP
Precision = 0.9947089947089947
Recall = 0.9894736842105263

ISTP
Precision = 0.9937578027465668
Recall = 0.9851485148514851

ISFP
Precision = 0.9915492957746479
Recall = 0.9846153846153847

INTP
Precision = 0.9871630295250321
Recall = 0.990979381443299

INFP
Precision = 0.9826435246

In [17]:
knn(4,1,False)  #   4.fold  ,  k = 1  ,  without feature normalization

ESTJ
Precision = 0.9822646657571623
Recall = 0.9795918367346939

ENTJ
Precision = 0.9745989304812834
Recall = 0.9811574697173621

ESFJ
Precision = 0.9788079470198675
Recall = 0.980106100795756

ENFJ
Precision = 0.9736180904522613
Recall = 0.9785353535353535

ISTJ
Precision = 0.9805825242718447
Recall = 0.9793939393939394

ISFJ
Precision = 0.9788359788359788
Recall = 0.9866666666666667

INTJ
Precision = 0.981081081081081
Recall = 0.981081081081081

INFJ
Precision = 0.9825503355704698
Recall = 0.9812332439678284

ESTP
Precision = 0.9803370786516854
Recall = 0.9789621318373072

ESFP
Precision = 0.9802538787023978
Recall = 0.9747545582047685

ENTP
Precision = 0.9838926174496644
Recall = 0.9838926174496644

ENFP
Precision = 0.9825268817204301
Recall = 0.9759679572763685

ISTP
Precision = 0.9804469273743017
Recall = 0.9804469273743017

ISFP
Precision = 0.9816272965879265
Recall = 0.9726918075422627

INTP
Precision = 0.9743243243243244
Recall = 0.9756427604871448

INFP
Precision = 0.967700258

In [18]:
knn(4,3,False)  #   4.fold  ,  k = 3  ,  without feature normalization

ESTJ
Precision = 0.9851351351351352
Recall = 0.9918367346938776

ENTJ
Precision = 0.9879518072289156
Recall = 0.9932705248990579

ESFJ
Precision = 0.9854689564068693
Recall = 0.9893899204244032

ENFJ
Precision = 0.9861286254728878
Recall = 0.9873737373737373

ISTJ
Precision = 0.9926829268292683
Recall = 0.9866666666666667

ISFJ
Precision = 0.9880478087649402
Recall = 0.992

INTJ
Precision = 0.9864864864864865
Recall = 0.9864864864864865

INFJ
Precision = 0.9892761394101877
Recall = 0.9892761394101877

ESTP
Precision = 0.9887640449438202
Recall = 0.9873772791023843

ESFP
Precision = 0.9957386363636364
Recall = 0.9831697054698457

ENTP
Precision = 0.989247311827957
Recall = 0.9879194630872483

ENFP
Precision = 0.989247311827957
Recall = 0.9826435246995995

ISTP
Precision = 0.9860335195530726
Recall = 0.9860335195530726

ISFP
Precision = 0.9870298313878081
Recall = 0.9895968790637191

INTP
Precision = 0.9877551020408163
Recall = 0.9824086603518268

INFP
Precision = 0.9742599742599742
Reca

In [19]:
knn(4,5,False)  #   4.fold  ,  k = 5  ,  without feature normalization

ESTJ
Precision = 0.986468200270636
Recall = 0.9918367346938776

ENTJ
Precision = 0.9892761394101877
Recall = 0.9932705248990579

ESFJ
Precision = 0.9894039735099338
Recall = 0.9907161803713528

ENFJ
Precision = 0.987389659520807
Recall = 0.9886363636363636

ISTJ
Precision = 0.9939024390243902
Recall = 0.9878787878787879

ISFJ
Precision = 0.9893899204244032
Recall = 0.9946666666666667

INTJ
Precision = 0.989159891598916
Recall = 0.9864864864864865

INFJ
Precision = 0.9906040268456375
Recall = 0.9892761394101877

ESTP
Precision = 0.9901685393258427
Recall = 0.9887798036465638

ESFP
Precision = 0.9957446808510638
Recall = 0.9845722300140253

ENTP
Precision = 0.989247311827957
Recall = 0.9879194630872483

ENFP
Precision = 0.9906040268456375
Recall = 0.9853137516688919

ISTP
Precision = 0.986013986013986
Recall = 0.9846368715083799

ISFP
Precision = 0.9870633893919794
Recall = 0.9921976592977894

INTP
Precision = 0.9878048780487805
Recall = 0.986468200270636

INFP
Precision = 0.976833976833

In [20]:
knn(4,7,False)  #   4.fold  ,  k = 7  ,  without feature normalization

ESTJ
Precision = 0.9878048780487805
Recall = 0.9918367346938776

ENTJ
Precision = 0.9892761394101877
Recall = 0.9932705248990579

ESFJ
Precision = 0.9894039735099338
Recall = 0.9907161803713528

ENFJ
Precision = 0.987389659520807
Recall = 0.9886363636363636

ISTJ
Precision = 0.9951159951159951
Recall = 0.9878787878787879

ISFJ
Precision = 0.9893899204244032
Recall = 0.9946666666666667

INTJ
Precision = 0.9878213802435724
Recall = 0.9864864864864865

INFJ
Precision = 0.9906040268456375
Recall = 0.9892761394101877

ESTP
Precision = 0.9901685393258427
Recall = 0.9887798036465638

ESFP
Precision = 0.9957446808510638
Recall = 0.9845722300140253

ENTP
Precision = 0.9892617449664429
Recall = 0.9892617449664429

ENFP
Precision = 0.9906040268456375
Recall = 0.9853137516688919

ISTP
Precision = 0.9860529986052998
Recall = 0.9874301675977654

ISFP
Precision = 0.9883419689119171
Recall = 0.9921976592977894

INTP
Precision = 0.9877883310719131
Recall = 0.9851150202976996

INFP
Precision = 0.9768339

In [21]:
knn(4,9,False)  #   4.fold  ,  k = 9  ,  without feature normalization

ESTJ
Precision = 0.9878048780487805
Recall = 0.9918367346938776

ENTJ
Precision = 0.9892761394101877
Recall = 0.9932705248990579

ESFJ
Precision = 0.9894039735099338
Recall = 0.9907161803713528

ENFJ
Precision = 0.987389659520807
Recall = 0.9886363636363636

ISTJ
Precision = 0.9939024390243902
Recall = 0.9878787878787879

ISFJ
Precision = 0.9893758300132802
Recall = 0.9933333333333333

INTJ
Precision = 0.9878213802435724
Recall = 0.9864864864864865

INFJ
Precision = 0.9906040268456375
Recall = 0.9892761394101877

ESTP
Precision = 0.9901685393258427
Recall = 0.9887798036465638

ESFP
Precision = 0.9957446808510638
Recall = 0.9845722300140253

ENTP
Precision = 0.9892617449664429
Recall = 0.9892617449664429

ENFP
Precision = 0.9906040268456375
Recall = 0.9853137516688919

ISTP
Precision = 0.9846582984658299
Recall = 0.9860335195530726

ISFP
Precision = 0.9883419689119171
Recall = 0.9921976592977894

INTP
Precision = 0.9877883310719131
Recall = 0.9851150202976996

INFP
Precision = 0.9768339

In [22]:
knn(5,1,False)  #   5.fold  ,  k = 1  ,  without feature normalization

ESTJ
Precision = 0.9879679144385026
Recall = 0.9814077025232404

ENTJ
Precision = 0.972936400541272
Recall = 0.9835841313269493

ESFJ
Precision = 0.982620320855615
Recall = 0.9722222222222222

ENFJ
Precision = 0.9769959404600812
Recall = 0.978319783197832

ISTJ
Precision = 0.983957219251337
Recall = 0.9787234042553191

ISFJ
Precision = 0.9797843665768194
Recall = 0.9732262382864793

INTJ
Precision = 0.9692737430167597
Recall = 0.9719887955182073

INFJ
Precision = 0.9864314789687924
Recall = 0.9784656796769852

ESTP
Precision = 0.9818417639429312
Recall = 0.9831168831168832

ESFP
Precision = 0.9765625
Recall = 0.984251968503937

ENTP
Precision = 0.9868247694334651
Recall = 0.9778067885117493

ENFP
Precision = 0.9717223650385605
Recall = 0.984375

ISTP
Precision = 0.9828947368421053
Recall = 0.9841897233201581

ISFP
Precision = 0.9651006711409396
Recall = 0.9690026954177897

INTP
Precision = 0.9649122807017544
Recall = 0.9701492537313433

INFP
Precision = 0.9725130890052356
Recall = 0.97

In [23]:
knn(5,3,False)  #   5.fold  ,  k = 3  ,  without feature normalization

ESTJ
Precision = 0.9973190348525469
Recall = 0.9880478087649402

ENTJ
Precision = 0.9836512261580381
Recall = 0.987688098495212

ESFJ
Precision = 0.9841059602649007
Recall = 0.9828042328042328

ENFJ
Precision = 0.9850746268656716
Recall = 0.983739837398374

ISTJ
Precision = 0.9906914893617021
Recall = 0.9906914893617021

ISFJ
Precision = 0.9919246298788694
Recall = 0.9866131191432396

INTJ
Precision = 0.9901823281907434
Recall = 0.988795518207283

INFJ
Precision = 0.9918256130790191
Recall = 0.9798115746971736

ESTP
Precision = 0.993514915693904
Recall = 0.9948051948051948

ESFP
Precision = 0.9856020942408377
Recall = 0.9881889763779528

ENTP
Precision = 0.9934469200524246
Recall = 0.9895561357702349

ENFP
Precision = 0.9883268482490273
Recall = 0.9921875

ISTP
Precision = 0.994729907773386
Recall = 0.994729907773386

ISFP
Precision = 0.9799732977303071
Recall = 0.9892183288409704

INTP
Precision = 0.9771197846567967
Recall = 0.9850746268656716

INFP
Precision = 0.9843953185955787
Reca

In [24]:
knn(5,5,False)  #   5.fold  ,  k = 5  ,  without feature normalization

ESTJ
Precision = 0.9973190348525469
Recall = 0.9880478087649402

ENTJ
Precision = 0.984993178717599
Recall = 0.987688098495212

ESFJ
Precision = 0.9893475366178429
Recall = 0.9828042328042328

ENFJ
Precision = 0.9850746268656716
Recall = 0.983739837398374

ISTJ
Precision = 0.9933333333333333
Recall = 0.9906914893617021

ISFJ
Precision = 0.9919246298788694
Recall = 0.9866131191432396

INTJ
Precision = 0.9888111888111888
Recall = 0.9901960784313726

INFJ
Precision = 0.9918588873812755
Recall = 0.9838492597577388

ESTP
Precision = 0.9922077922077922
Recall = 0.9922077922077922

ESFP
Precision = 0.9856020942408377
Recall = 0.9881889763779528

ENTP
Precision = 0.9934469200524246
Recall = 0.9895561357702349

ENFP
Precision = 0.9896103896103896
Recall = 0.9921875

ISTP
Precision = 0.994729907773386
Recall = 0.994729907773386

ISFP
Precision = 0.9813333333333333
Recall = 0.9919137466307277

INTP
Precision = 0.9772117962466488
Recall = 0.989145183175034

INFP
Precision = 0.9843953185955787
Reca

In [25]:
knn(5,7,False)  #   5.fold  ,  k = 7  ,  without feature normalization

ESTJ
Precision = 0.9973190348525469
Recall = 0.9880478087649402

ENTJ
Precision = 0.984993178717599
Recall = 0.987688098495212

ESFJ
Precision = 0.9893475366178429
Recall = 0.9828042328042328

ENFJ
Precision = 0.983739837398374
Recall = 0.983739837398374

ISTJ
Precision = 0.9933244325767691
Recall = 0.9893617021276596

ISFJ
Precision = 0.9919246298788694
Recall = 0.9866131191432396

INTJ
Precision = 0.9888111888111888
Recall = 0.9901960784313726

INFJ
Precision = 0.9905149051490515
Recall = 0.9838492597577388

ESTP
Precision = 0.9922178988326849
Recall = 0.9935064935064936

ESFP
Precision = 0.9856020942408377
Recall = 0.9881889763779528

ENTP
Precision = 0.9934469200524246
Recall = 0.9895561357702349

ENFP
Precision = 0.9896103896103896
Recall = 0.9921875

ISTP
Precision = 0.9947229551451188
Recall = 0.9934123847167325

ISFP
Precision = 0.9813333333333333
Recall = 0.9919137466307277

INTP
Precision = 0.9772117962466488
Recall = 0.989145183175034

INFP
Precision = 0.9856770833333334
Rec

In [26]:
knn(5,9,False)  #   5.fold  ,  k = 9  ,  without feature normalization

ESTJ
Precision = 0.9986577181208054
Recall = 0.9880478087649402

ENTJ
Precision = 0.984993178717599
Recall = 0.987688098495212

ESFJ
Precision = 0.9906666666666667
Recall = 0.9828042328042328

ENFJ
Precision = 0.9864130434782609
Recall = 0.983739837398374

ISTJ
Precision = 0.9933244325767691
Recall = 0.9893617021276596

ISFJ
Precision = 0.9919463087248322
Recall = 0.9892904953145917

INTJ
Precision = 0.9874476987447699
Recall = 0.9915966386554622

INFJ
Precision = 0.9905149051490515
Recall = 0.9838492597577388

ESTP
Precision = 0.9922178988326849
Recall = 0.9935064935064936

ESFP
Precision = 0.9855832241153342
Recall = 0.9868766404199475

ENTP
Precision = 0.9934469200524246
Recall = 0.9895561357702349

ENFP
Precision = 0.9883268482490273
Recall = 0.9921875

ISTP
Precision = 0.994729907773386
Recall = 0.994729907773386

ISFP
Precision = 0.9813333333333333
Recall = 0.9919137466307277

INTP
Precision = 0.978523489932886
Recall = 0.989145183175034

INFP
Precision = 0.9856770833333334
Recal

In [27]:
knn(1,1,True)  #   1.fold  ,  k = 1  ,  with feature normalization

ESTJ
Precision = 0.9724047306176085
Recall = 0.9762532981530343

ENTJ
Precision = 0.979746835443038
Recall = 0.979746835443038

ESFJ
Precision = 0.9813084112149533
Recall = 0.9865771812080537

ENFJ
Precision = 0.970310391363023
Recall = 0.9876373626373627

ISTJ
Precision = 0.9755102040816327
Recall = 0.9768392370572208

ISFJ
Precision = 0.9755784061696658
Recall = 0.9793548387096774

INTJ
Precision = 0.9920318725099602
Recall = 0.9790301441677588

INFJ
Precision = 0.9731800766283525
Recall = 0.9883268482490273

ESTP
Precision = 0.9826435246995995
Recall = 0.9852744310575636

ESFP
Precision = 0.9755102040816327
Recall = 0.9741847826086957

ENTP
Precision = 0.9878542510121457
Recall = 0.973404255319149

ENFP
Precision = 0.9833101529902643
Recall = 0.9711538461538461

ISTP
Precision = 0.9795918367346939
Recall = 0.9795918367346939

ISFP
Precision = 0.984313725490196
Recall = 0.9817470664928292

INTP
Precision = 0.9768076398362893
Recall = 0.9715061058344641

INFP
Precision = 0.97131147540

In [28]:
knn(1,3,True)  #   1.fold  ,  k = 3  ,  with feature normalization

ESTJ
Precision = 0.9880478087649402
Recall = 0.9815303430079155

ENTJ
Precision = 0.9824561403508771
Recall = 0.9924050632911392

ESFJ
Precision = 0.9801324503311258
Recall = 0.9932885906040269

ENFJ
Precision = 0.9837177747625508
Recall = 0.9958791208791209

ISTJ
Precision = 0.9864130434782609
Recall = 0.989100817438692

ISFJ
Precision = 0.9846153846153847
Recall = 0.9909677419354839

INTJ
Precision = 0.9960264900662251
Recall = 0.9855832241153342

INFJ
Precision = 0.982051282051282
Recall = 0.993514915693904

ESTP
Precision = 0.9893048128342246
Recall = 0.9906291834002677

ESFP
Precision = 0.989010989010989
Recall = 0.9782608695652174

ENTP
Precision = 0.9973154362416108
Recall = 0.988031914893617

ENFP
Precision = 0.9930843706777317
Recall = 0.9862637362637363

ISTP
Precision = 0.9877049180327869
Recall = 0.9836734693877551

ISFP
Precision = 0.9909208819714657
Recall = 0.9960886571056062

INTP
Precision = 0.9918032786885246
Recall = 0.9850746268656716

INFP
Precision = 0.99449035812

In [29]:
knn(1,5,True)  #   1.fold  ,  k = 5  ,  with feature normalization

ESTJ
Precision = 0.9880636604774535
Recall = 0.9828496042216359

ENTJ
Precision = 0.9861635220125786
Recall = 0.9924050632911392

ESFJ
Precision = 0.9801324503311258
Recall = 0.9932885906040269

ENFJ
Precision = 0.9890859481582538
Recall = 0.9958791208791209

ISTJ
Precision = 0.9877384196185286
Recall = 0.9877384196185286

ISFJ
Precision = 0.9871630295250321
Recall = 0.9922580645161291

INTJ
Precision = 0.9934123847167325
Recall = 0.9882044560943644

INFJ
Precision = 0.9833119383825417
Recall = 0.993514915693904

ESTP
Precision = 0.9892904953145917
Recall = 0.9892904953145917

ESFP
Precision = 0.9890260631001372
Recall = 0.9796195652173914

ENTP
Precision = 0.9973154362416108
Recall = 0.988031914893617

ENFP
Precision = 0.9930939226519337
Recall = 0.9876373626373627

ISTP
Precision = 0.9877216916780355
Recall = 0.9850340136054422

ISFP
Precision = 0.9922077922077922
Recall = 0.9960886571056062

INTP
Precision = 0.9904761904761905
Recall = 0.9877883310719131

INFP
Precision = 0.99313186

In [30]:
knn(1,7,True)  #   1.fold  ,  k = 7  ,  with feature normalization

ESTJ
Precision = 0.9880636604774535
Recall = 0.9828496042216359

ENTJ
Precision = 0.9861635220125786
Recall = 0.9924050632911392

ESFJ
Precision = 0.9801324503311258
Recall = 0.9932885906040269

ENFJ
Precision = 0.9890859481582538
Recall = 0.9958791208791209

ISTJ
Precision = 0.9877384196185286
Recall = 0.9877384196185286

ISFJ
Precision = 0.9871630295250321
Recall = 0.9922580645161291

INTJ
Precision = 0.9947229551451188
Recall = 0.9882044560943644

INFJ
Precision = 0.9845758354755784
Recall = 0.993514915693904

ESTP
Precision = 0.9906291834002677
Recall = 0.9906291834002677

ESFP
Precision = 0.987688098495212
Recall = 0.9809782608695652

ENTP
Precision = 0.9973154362416108
Recall = 0.988031914893617

ENFP
Precision = 0.9930843706777317
Recall = 0.9862637362637363

ISTP
Precision = 0.9877384196185286
Recall = 0.9863945578231292

ISFP
Precision = 0.9934980494148244
Recall = 0.9960886571056062

INTP
Precision = 0.9904891304347826
Recall = 0.989145183175034

INFP
Precision = 0.9931318681

In [31]:
knn(1,9,True)  #   1.fold  ,  k = 9  ,  with feature normalization

ESTJ
Precision = 0.9880636604774535
Recall = 0.9828496042216359

ENTJ
Precision = 0.9861635220125786
Recall = 0.9924050632911392

ESFJ
Precision = 0.9801324503311258
Recall = 0.9932885906040269

ENFJ
Precision = 0.9890859481582538
Recall = 0.9958791208791209

ISTJ
Precision = 0.9877384196185286
Recall = 0.9877384196185286

ISFJ
Precision = 0.9884318766066839
Recall = 0.9922580645161291

INTJ
Precision = 0.994729907773386
Recall = 0.9895150720838795

INFJ
Precision = 0.9845758354755784
Recall = 0.993514915693904

ESTP
Precision = 0.9906291834002677
Recall = 0.9906291834002677

ESFP
Precision = 0.989041095890411
Recall = 0.9809782608695652

ENTP
Precision = 0.9973154362416108
Recall = 0.988031914893617

ENFP
Precision = 0.9930939226519337
Recall = 0.9876373626373627

ISTP
Precision = 0.9877551020408163
Recall = 0.9877551020408163

ISFP
Precision = 0.9934980494148244
Recall = 0.9960886571056062

INTP
Precision = 0.9904761904761905
Recall = 0.9877883310719131

INFP
Precision = 0.9931318681

In [32]:
knn(2,1,True)  #   2.fold  ,  k = 1  ,  with feature normalization

ESTJ
Precision = 0.9811083123425692
Recall = 0.9811083123425692

ENTJ
Precision = 0.97265625
Recall = 0.9841897233201581

ESFJ
Precision = 0.9772727272727273
Recall = 0.9759679572763685

ENFJ
Precision = 0.9739583333333334
Recall = 0.9829172141918529

ISTJ
Precision = 0.9713114754098361
Recall = 0.9779917469050894

ISFJ
Precision = 0.9791666666666666
Recall = 0.9835575485799701

INTJ
Precision = 0.9661654135338346
Recall = 0.9834183673469388

INFJ
Precision = 0.988031914893617
Recall = 0.9815059445178336

ESTP
Precision = 0.9827814569536424
Recall = 0.9840848806366048

ESFP
Precision = 0.9847328244274809
Recall = 0.9772727272727273

ENTP
Precision = 0.9635135135135136
Recall = 0.9767123287671233

ENFP
Precision = 0.9775725593667546
Recall = 0.9814569536423841

ISTP
Precision = 0.9768392370572208
Recall = 0.9728629579375848

ISFP
Precision = 0.9850746268656716
Recall = 0.9641434262948207

INTP
Precision = 0.972972972972973
Recall = 0.9561752988047809

INFP
Precision = 0.9748953974895398

In [33]:
knn(2,3,True)  #   2.fold  ,  k = 3  ,  with feature normalization

ESTJ
Precision = 0.9886792452830189
Recall = 0.9899244332493703

ENTJ
Precision = 0.9933862433862434
Recall = 0.9894598155467721

ESFJ
Precision = 0.9827814569536424
Recall = 0.9906542056074766

ENFJ
Precision = 0.989501312335958
Recall = 0.9908015768725361

ISTJ
Precision = 0.9889958734525447
Recall = 0.9889958734525447

ISFJ
Precision = 0.9880597014925373
Recall = 0.9895366218236173

INTJ
Precision = 0.9823455233291298
Recall = 0.9936224489795918

INFJ
Precision = 0.9947019867549669
Recall = 0.9920739762219286

ESTP
Precision = 0.9907161803713528
Recall = 0.9907161803713528

ESFP
Precision = 0.9961783439490446
Recall = 0.9873737373737373

ENTP
Precision = 0.984993178717599
Recall = 0.989041095890411

ENFP
Precision = 0.9893758300132802
Recall = 0.9867549668874173

ISTP
Precision = 0.9904632152588556
Recall = 0.9864314789687924

ISFP
Precision = 0.9906166219839142
Recall = 0.9814077025232404

INTP
Precision = 0.9853528628495339
Recall = 0.9827357237715804

INFP
Precision = 0.986301369

In [34]:
knn(2,5,True)  #   2.fold  ,  k = 5  ,  with feature normalization

ESTJ
Precision = 0.9899244332493703
Recall = 0.9899244332493703

ENTJ
Precision = 0.9947019867549669
Recall = 0.9894598155467721

ESFJ
Precision = 0.9828042328042328
Recall = 0.9919893190921228

ENFJ
Precision = 0.989501312335958
Recall = 0.9908015768725361

ISTJ
Precision = 0.990358126721763
Recall = 0.9889958734525447

ISFJ
Precision = 0.9880418535127056
Recall = 0.9880418535127056

INTJ
Precision = 0.9823455233291298
Recall = 0.9936224489795918

INFJ
Precision = 0.9947019867549669
Recall = 0.9920739762219286

ESTP
Precision = 0.9894039735099338
Recall = 0.9907161803713528

ESFP
Precision = 0.9961783439490446
Recall = 0.9873737373737373

ENTP
Precision = 0.9836734693877551
Recall = 0.9904109589041096

ENFP
Precision = 0.9893758300132802
Recall = 0.9867549668874173

ISTP
Precision = 0.9917920656634747
Recall = 0.9837177747625508

ISFP
Precision = 0.9906417112299465
Recall = 0.9840637450199203

INTP
Precision = 0.9853528628495339
Recall = 0.9827357237715804

INFP
Precision = 0.98632010

In [35]:
knn(2,7,True)  #   2.fold  ,  k = 7  ,  with feature normalization

ESTJ
Precision = 0.9911727616645649
Recall = 0.9899244332493703

ENTJ
Precision = 0.9947019867549669
Recall = 0.9894598155467721

ESFJ
Precision = 0.9802110817941952
Recall = 0.9919893190921228

ENFJ
Precision = 0.9895150720838795
Recall = 0.9921156373193167

ISTJ
Precision = 0.990358126721763
Recall = 0.9889958734525447

ISFJ
Precision = 0.9880418535127056
Recall = 0.9880418535127056

INTJ
Precision = 0.9848484848484849
Recall = 0.9948979591836735

INFJ
Precision = 0.9947019867549669
Recall = 0.9920739762219286

ESTP
Precision = 0.9906914893617021
Recall = 0.9880636604774535

ESFP
Precision = 0.9949109414758269
Recall = 0.9873737373737373

ENTP
Precision = 0.9850136239782016
Recall = 0.9904109589041096

ENFP
Precision = 0.9893758300132802
Recall = 0.9867549668874173

ISTP
Precision = 0.9918032786885246
Recall = 0.9850746268656716

ISFP
Precision = 0.9906417112299465
Recall = 0.9840637450199203

INTP
Precision = 0.9853528628495339
Recall = 0.9827357237715804

INFP
Precision = 0.9849726

In [36]:
knn(2,9,True)  #   2.fold  ,  k = 9  ,  with feature normalization

ESTJ
Precision = 0.9911727616645649
Recall = 0.9899244332493703

ENTJ
Precision = 0.9947019867549669
Recall = 0.9894598155467721

ESFJ
Precision = 0.9828042328042328
Recall = 0.9919893190921228

ENFJ
Precision = 0.9895150720838795
Recall = 0.9921156373193167

ISTJ
Precision = 0.990358126721763
Recall = 0.9889958734525447

ISFJ
Precision = 0.9880597014925373
Recall = 0.9895366218236173

INTJ
Precision = 0.9848293299620733
Recall = 0.9936224489795918

INFJ
Precision = 0.9947019867549669
Recall = 0.9920739762219286

ESTP
Precision = 0.9894179894179894
Recall = 0.9920424403183024

ESFP
Precision = 0.9961783439490446
Recall = 0.9873737373737373

ENTP
Precision = 0.9850136239782016
Recall = 0.9904109589041096

ENFP
Precision = 0.9893758300132802
Recall = 0.9867549668874173

ISTP
Precision = 0.9918144611186903
Recall = 0.9864314789687924

ISFP
Precision = 0.9906417112299465
Recall = 0.9840637450199203

INTP
Precision = 0.9853528628495339
Recall = 0.9827357237715804

INFP
Precision = 0.9876712

In [37]:
knn(3,1,True)  #   3.fold  ,  k = 1  ,  with feature normalization

ESTJ
Precision = 0.9833101529902643
Recall = 0.9833101529902643

ENTJ
Precision = 0.9682758620689655
Recall = 0.9831932773109243

ESFJ
Precision = 0.9836734693877551
Recall = 0.9743935309973046

ENFJ
Precision = 0.9793672627235214
Recall = 0.9834254143646409

ISTJ
Precision = 0.9777777777777777
Recall = 0.9805013927576601

ISFJ
Precision = 0.9702233250620348
Recall = 0.9799498746867168

INTJ
Precision = 0.966711051930759
Recall = 0.9784366576819407

INFJ
Precision = 0.991701244813278
Recall = 0.9637096774193549

ESTP
Precision = 0.9803407601572739
Recall = 0.9777777777777777

ESFP
Precision = 0.9777195281782438
Recall = 0.9738903394255874

ENTP
Precision = 0.9740596627756161
Recall = 0.9791395045632334

ENFP
Precision = 0.9790575916230366
Recall = 0.9842105263157894

ISTP
Precision = 0.9801488833746899
Recall = 0.9777227722772277

ISFP
Precision = 0.9817671809256662
Recall = 0.9790209790209791

INTP
Precision = 0.9778645833333334
Recall = 0.9677835051546392

INFP
Precision = 0.96241610

In [38]:
knn(3,3,True)  #   3.fold  ,  k = 3  ,  with feature normalization

ESTJ
Precision = 0.9944289693593314
Recall = 0.9930458970792768

ENTJ
Precision = 0.9820193637621023
Recall = 0.9943977591036415

ESFJ
Precision = 0.9905277401894452
Recall = 0.9865229110512129

ENFJ
Precision = 0.9862825788751715
Recall = 0.9930939226519337

ISTJ
Precision = 0.9874301675977654
Recall = 0.9846796657381616

ISFJ
Precision = 0.9813432835820896
Recall = 0.9887218045112782

INTJ
Precision = 0.9826435246995995
Recall = 0.9919137466307277

INFJ
Precision = 0.9986357435197817
Recall = 0.9838709677419355

ESTP
Precision = 0.9895561357702349
Recall = 0.9908496732026144

ESFP
Precision = 0.9908015768725361
Recall = 0.9843342036553525

ENTP
Precision = 0.9921465968586387
Recall = 0.9882659713168188

ENFP
Precision = 0.9947019867549669
Recall = 0.9881578947368421

ISTP
Precision = 0.9925093632958801
Recall = 0.9839108910891089

ISFP
Precision = 0.9901547116736991
Recall = 0.9846153846153847

INTP
Precision = 0.985897435897436
Recall = 0.990979381443299

INFP
Precision = 0.98133333

In [39]:
knn(3,5,True)  #   3.fold  ,  k = 5  ,  with feature normalization

ESTJ
Precision = 0.9944367176634215
Recall = 0.9944367176634215

ENTJ
Precision = 0.9861111111111112
Recall = 0.9943977591036415

ESFJ
Precision = 0.9905405405405405
Recall = 0.9878706199460916

ENFJ
Precision = 0.9876373626373627
Recall = 0.9930939226519337

ISTJ
Precision = 0.9888268156424581
Recall = 0.9860724233983287

ISFJ
Precision = 0.9825653798256538
Recall = 0.9887218045112782

INTJ
Precision = 0.9813084112149533
Recall = 0.9905660377358491

INFJ
Precision = 0.998639455782313
Recall = 0.9865591397849462

ESTP
Precision = 0.9908496732026144
Recall = 0.9908496732026144

ESFP
Precision = 0.9908136482939632
Recall = 0.9856396866840731

ENTP
Precision = 0.9908496732026144
Recall = 0.9882659713168188

ENFP
Precision = 0.9947019867549669
Recall = 0.9881578947368421

ISTP
Precision = 0.9925187032418953
Recall = 0.9851485148514851

ISFP
Precision = 0.9915492957746479
Recall = 0.9846153846153847

INTP
Precision = 0.9871630295250321
Recall = 0.990979381443299

INFP
Precision = 0.98002663

In [40]:
knn(3,7,True)  #   3.fold  ,  k = 7  ,  with feature normalization

ESTJ
Precision = 0.9944367176634215
Recall = 0.9944367176634215

ENTJ
Precision = 0.9874826147426982
Recall = 0.9943977591036415

ESFJ
Precision = 0.99055330634278
Recall = 0.9892183288409704

ENFJ
Precision = 0.9889958734525447
Recall = 0.9930939226519337

ISTJ
Precision = 0.9888579387186629
Recall = 0.9888579387186629

ISFJ
Precision = 0.9838107098381071
Recall = 0.9899749373433584

INTJ
Precision = 0.9813581890812251
Recall = 0.9932614555256065

INFJ
Precision = 0.998639455782313
Recall = 0.9865591397849462

ESTP
Precision = 0.9908496732026144
Recall = 0.9908496732026144

ESFP
Precision = 0.9921156373193167
Recall = 0.9856396866840731

ENTP
Precision = 0.9921465968586387
Recall = 0.9882659713168188

ENFP
Precision = 0.9947019867549669
Recall = 0.9881578947368421

ISTP
Precision = 0.9937578027465668
Recall = 0.9851485148514851

ISFP
Precision = 0.9915492957746479
Recall = 0.9846153846153847

INTP
Precision = 0.9871630295250321
Recall = 0.990979381443299

INFP
Precision = 0.9813581890

In [41]:
knn(3,9,True)  #   3.fold  ,  k = 9  ,  with feature normalization

ESTJ
Precision = 0.9944444444444445
Recall = 0.9958275382475661

ENTJ
Precision = 0.9874826147426982
Recall = 0.9943977591036415

ESFJ
Precision = 0.99055330634278
Recall = 0.9892183288409704

ENFJ
Precision = 0.9889958734525447
Recall = 0.9930939226519337

ISTJ
Precision = 0.9888579387186629
Recall = 0.9888579387186629

ISFJ
Precision = 0.9838107098381071
Recall = 0.9899749373433584

INTJ
Precision = 0.9813581890812251
Recall = 0.9932614555256065

INFJ
Precision = 0.998639455782313
Recall = 0.9865591397849462

ESTP
Precision = 0.9908376963350786
Recall = 0.9895424836601308

ESFP
Precision = 0.9908136482939632
Recall = 0.9856396866840731

ENTP
Precision = 0.9921465968586387
Recall = 0.9882659713168188

ENFP
Precision = 0.9947089947089947
Recall = 0.9894736842105263

ISTP
Precision = 0.9937578027465668
Recall = 0.9851485148514851

ISFP
Precision = 0.9915492957746479
Recall = 0.9846153846153847

INTP
Precision = 0.9871630295250321
Recall = 0.990979381443299

INFP
Precision = 0.9826435246

In [42]:
knn(4,1,True)  #   4.fold  ,  k = 1  ,  with feature normalization

ESTJ
Precision = 0.9822888283378747
Recall = 0.9809523809523809

ENTJ
Precision = 0.9732977303070761
Recall = 0.9811574697173621

ESFJ
Precision = 0.9787798408488063
Recall = 0.9787798408488063

ENFJ
Precision = 0.9748427672955975
Recall = 0.9785353535353535

ISTJ
Precision = 0.9817961165048543
Recall = 0.9806060606060606

ISFJ
Precision = 0.9788359788359788
Recall = 0.9866666666666667

INTJ
Precision = 0.9797297297297297
Recall = 0.9797297297297297

INFJ
Precision = 0.9838709677419355
Recall = 0.9812332439678284

ESTP
Precision = 0.9789029535864979
Recall = 0.9761570827489481

ESFP
Precision = 0.9802816901408451
Recall = 0.9761570827489481

ENTP
Precision = 0.982573726541555
Recall = 0.9838926174496644

ENFP
Precision = 0.9812080536912752
Recall = 0.9759679572763685

ISTP
Precision = 0.9818181818181818
Recall = 0.9804469273743017

ISFP
Precision = 0.9803664921465969
Recall = 0.9739921976592978

INTP
Precision = 0.9743243243243244
Recall = 0.9756427604871448

INFP
Precision = 0.9689119

In [43]:
knn(4,3,True)  #   4.fold  ,  k = 3  ,  with feature normalization

ESTJ
Precision = 0.986468200270636
Recall = 0.9918367346938776

ENTJ
Precision = 0.9879518072289156
Recall = 0.9932705248990579

ESFJ
Precision = 0.9841688654353562
Recall = 0.9893899204244032

ENFJ
Precision = 0.9861286254728878
Recall = 0.9873737373737373

ISTJ
Precision = 0.9926829268292683
Recall = 0.9866666666666667

ISFJ
Precision = 0.9880636604774535
Recall = 0.9933333333333333

INTJ
Precision = 0.9878213802435724
Recall = 0.9864864864864865

INFJ
Precision = 0.9892761394101877
Recall = 0.9892761394101877

ESTP
Precision = 0.9887640449438202
Recall = 0.9873772791023843

ESFP
Precision = 0.9957386363636364
Recall = 0.9831697054698457

ENTP
Precision = 0.989247311827957
Recall = 0.9879194630872483

ENFP
Precision = 0.989247311827957
Recall = 0.9826435246995995

ISTP
Precision = 0.9860335195530726
Recall = 0.9860335195530726

ISFP
Precision = 0.9870466321243523
Recall = 0.9908972691807543

INTP
Precision = 0.9877551020408163
Recall = 0.9824086603518268

INFP
Precision = 0.975515463

In [44]:
knn(4,5,True)  #   4.fold  ,  k = 1  ,  with feature normalization

ESTJ
Precision = 0.9878048780487805
Recall = 0.9918367346938776

ENTJ
Precision = 0.9892761394101877
Recall = 0.9932705248990579

ESFJ
Precision = 0.9894039735099338
Recall = 0.9907161803713528

ENFJ
Precision = 0.987389659520807
Recall = 0.9886363636363636

ISTJ
Precision = 0.9926918392204629
Recall = 0.9878787878787879

ISFJ
Precision = 0.9880794701986755
Recall = 0.9946666666666667

INTJ
Precision = 0.989159891598916
Recall = 0.9864864864864865

INFJ
Precision = 0.9906040268456375
Recall = 0.9892761394101877

ESTP
Precision = 0.9901685393258427
Recall = 0.9887798036465638

ESFP
Precision = 0.9957446808510638
Recall = 0.9845722300140253

ENTP
Precision = 0.989247311827957
Recall = 0.9879194630872483

ENFP
Precision = 0.9905913978494624
Recall = 0.9839786381842457

ISTP
Precision = 0.9860335195530726
Recall = 0.9860335195530726

ISFP
Precision = 0.9857881136950905
Recall = 0.9921976592977894

INTP
Precision = 0.9877717391304348
Recall = 0.9837618403247632

INFP
Precision = 0.976833976

In [45]:
knn(4,7,True)  #   4.fold  ,  k = 7  ,  with feature normalization

ESTJ
Precision = 0.9878048780487805
Recall = 0.9918367346938776

ENTJ
Precision = 0.9892761394101877
Recall = 0.9932705248990579

ESFJ
Precision = 0.9894039735099338
Recall = 0.9907161803713528

ENFJ
Precision = 0.987389659520807
Recall = 0.9886363636363636

ISTJ
Precision = 0.9951159951159951
Recall = 0.9878787878787879

ISFJ
Precision = 0.9893899204244032
Recall = 0.9946666666666667

INTJ
Precision = 0.9878213802435724
Recall = 0.9864864864864865

INFJ
Precision = 0.9906040268456375
Recall = 0.9892761394101877

ESTP
Precision = 0.9901685393258427
Recall = 0.9887798036465638

ESFP
Precision = 0.9957446808510638
Recall = 0.9845722300140253

ENTP
Precision = 0.9892617449664429
Recall = 0.9892617449664429

ENFP
Precision = 0.9906040268456375
Recall = 0.9853137516688919

ISTP
Precision = 0.9860529986052998
Recall = 0.9874301675977654

ISFP
Precision = 0.9883419689119171
Recall = 0.9921976592977894

INTP
Precision = 0.9877883310719131
Recall = 0.9851150202976996

INFP
Precision = 0.9768339

In [46]:
knn(4,9,True)  #   4.fold  ,  k = 9  ,  with feature normalization

ESTJ
Precision = 0.9878048780487805
Recall = 0.9918367346938776

ENTJ
Precision = 0.9892761394101877
Recall = 0.9932705248990579

ESFJ
Precision = 0.9894039735099338
Recall = 0.9907161803713528

ENFJ
Precision = 0.987389659520807
Recall = 0.9886363636363636

ISTJ
Precision = 0.9939024390243902
Recall = 0.9878787878787879

ISFJ
Precision = 0.9893758300132802
Recall = 0.9933333333333333

INTJ
Precision = 0.9878213802435724
Recall = 0.9864864864864865

INFJ
Precision = 0.9906040268456375
Recall = 0.9892761394101877

ESTP
Precision = 0.9901685393258427
Recall = 0.9887798036465638

ESFP
Precision = 0.9957446808510638
Recall = 0.9845722300140253

ENTP
Precision = 0.9892617449664429
Recall = 0.9892617449664429

ENFP
Precision = 0.9906040268456375
Recall = 0.9853137516688919

ISTP
Precision = 0.9846582984658299
Recall = 0.9860335195530726

ISFP
Precision = 0.9883419689119171
Recall = 0.9921976592977894

INTP
Precision = 0.9877883310719131
Recall = 0.9851150202976996

INFP
Precision = 0.9768339

In [47]:
knn(5,1,True)  #   5.fold  ,  k = 1  ,  with feature normalization

ESTJ
Precision = 0.9906291834002677
Recall = 0.9827357237715804

ENTJ
Precision = 0.9728997289972899
Recall = 0.9822161422708618

ESFJ
Precision = 0.9813333333333333
Recall = 0.9735449735449735

ENFJ
Precision = 0.9769959404600812
Recall = 0.978319783197832

ISTJ
Precision = 0.9826666666666667
Recall = 0.9800531914893617

ISFJ
Precision = 0.9811320754716981
Recall = 0.9745649263721553

INTJ
Precision = 0.9706703910614525
Recall = 0.9733893557422969

INFJ
Precision = 0.9864314789687924
Recall = 0.9784656796769852

ESTP
Precision = 0.9818652849740933
Recall = 0.9844155844155844

ESFP
Precision = 0.9791395045632334
Recall = 0.9855643044619422

ENTP
Precision = 0.9881266490765171
Recall = 0.9778067885117493

ENFP
Precision = 0.9742599742599742
Recall = 0.9856770833333334

ISTP
Precision = 0.9828947368421053
Recall = 0.9841897233201581

ISFP
Precision = 0.9664879356568364
Recall = 0.9716981132075472

INTP
Precision = 0.9662618083670715
Recall = 0.9715061058344641

INFP
Precision = 0.9750982

In [48]:
knn(5,3,True)  #   5.fold  ,  k = 3  ,  with feature normalization

ESTJ
Precision = 0.9986577181208054
Recall = 0.9880478087649402

ENTJ
Precision = 0.9836512261580381
Recall = 0.987688098495212

ESFJ
Precision = 0.9841059602649007
Recall = 0.9828042328042328

ENFJ
Precision = 0.9850746268656716
Recall = 0.983739837398374

ISTJ
Precision = 0.9920106524633822
Recall = 0.9906914893617021

ISFJ
Precision = 0.9919354838709677
Recall = 0.9879518072289156

INTJ
Precision = 0.9901823281907434
Recall = 0.988795518207283

INFJ
Precision = 0.9918367346938776
Recall = 0.9811574697173621

ESTP
Precision = 0.993514915693904
Recall = 0.9948051948051948

ESFP
Precision = 0.9856209150326798
Recall = 0.989501312335958

ENTP
Precision = 0.9934469200524246
Recall = 0.9895561357702349

ENFP
Precision = 0.9883268482490273
Recall = 0.9921875

ISTP
Precision = 0.994729907773386
Recall = 0.994729907773386

ISFP
Precision = 0.9799732977303071
Recall = 0.9892183288409704

INTP
Precision = 0.9771197846567967
Recall = 0.9850746268656716

INFP
Precision = 0.9856770833333334
Recal

In [49]:
knn(5,5,True)  #   5.fold  ,  k = 5  ,  with feature normalization

ESTJ
Precision = 0.9973190348525469
Recall = 0.9880478087649402

ENTJ
Precision = 0.984993178717599
Recall = 0.987688098495212

ESFJ
Precision = 0.9893475366178429
Recall = 0.9828042328042328

ENFJ
Precision = 0.9850746268656716
Recall = 0.983739837398374

ISTJ
Precision = 0.9933244325767691
Recall = 0.9893617021276596

ISFJ
Precision = 0.9919246298788694
Recall = 0.9866131191432396

INTJ
Precision = 0.9888111888111888
Recall = 0.9901960784313726

INFJ
Precision = 0.9918588873812755
Recall = 0.9838492597577388

ESTP
Precision = 0.9922077922077922
Recall = 0.9922077922077922

ESFP
Precision = 0.9856020942408377
Recall = 0.9881889763779528

ENTP
Precision = 0.9934469200524246
Recall = 0.9895561357702349

ENFP
Precision = 0.9896103896103896
Recall = 0.9921875

ISTP
Precision = 0.994729907773386
Recall = 0.994729907773386

ISFP
Precision = 0.9813333333333333
Recall = 0.9919137466307277

INTP
Precision = 0.9759036144578314
Recall = 0.989145183175034

INFP
Precision = 0.9843953185955787
Reca

In [50]:
knn(5,7,True)  #   5.fold  ,  k = 7  ,  with feature normalization

ESTJ
Precision = 0.9986577181208054
Recall = 0.9880478087649402

ENTJ
Precision = 0.984993178717599
Recall = 0.987688098495212

ESFJ
Precision = 0.9893475366178429
Recall = 0.9828042328042328

ENFJ
Precision = 0.983739837398374
Recall = 0.983739837398374

ISTJ
Precision = 0.9933244325767691
Recall = 0.9893617021276596

ISFJ
Precision = 0.9919354838709677
Recall = 0.9879518072289156

INTJ
Precision = 0.9888111888111888
Recall = 0.9901960784313726

INFJ
Precision = 0.9905149051490515
Recall = 0.9838492597577388

ESTP
Precision = 0.9922178988326849
Recall = 0.9935064935064936

ESFP
Precision = 0.9856020942408377
Recall = 0.9881889763779528

ENTP
Precision = 0.9934469200524246
Recall = 0.9895561357702349

ENFP
Precision = 0.9896103896103896
Recall = 0.9921875

ISTP
Precision = 0.9947229551451188
Recall = 0.9934123847167325

ISFP
Precision = 0.9813333333333333
Recall = 0.9919137466307277

INTP
Precision = 0.9772117962466488
Recall = 0.989145183175034

INFP
Precision = 0.9856770833333334
Rec

In [51]:
knn(5,9,True)  #   5.fold  ,  k = 9  ,  with feature normalization

ESTJ
Precision = 0.9986577181208054
Recall = 0.9880478087649402

ENTJ
Precision = 0.984993178717599
Recall = 0.987688098495212

ESFJ
Precision = 0.9906666666666667
Recall = 0.9828042328042328

ENFJ
Precision = 0.9864130434782609
Recall = 0.983739837398374

ISTJ
Precision = 0.9933244325767691
Recall = 0.9893617021276596

ISFJ
Precision = 0.9919463087248322
Recall = 0.9892904953145917

INTJ
Precision = 0.9888268156424581
Recall = 0.9915966386554622

INFJ
Precision = 0.9905277401894452
Recall = 0.9851951547779273

ESTP
Precision = 0.9922178988326849
Recall = 0.9935064935064936

ESFP
Precision = 0.9855832241153342
Recall = 0.9868766404199475

ENTP
Precision = 0.9934469200524246
Recall = 0.9895561357702349

ENFP
Precision = 0.9883268482490273
Recall = 0.9921875

ISTP
Precision = 0.994729907773386
Recall = 0.994729907773386

ISFP
Precision = 0.9813333333333333
Recall = 0.9919137466307277

INTP
Precision = 0.978523489932886
Recall = 0.989145183175034

INFP
Precision = 0.9856770833333334
Recal

In [ ]:
"""
ERROR ANALYSIS FOR CLASSIFICATION

We can observe that our prediction is most accurate when our k (in k nearest neighbour) is 5
Also with feature normalization, our prediction getting more accurate
Changing the k-fold does not give straight answers to us
We are not getting any misclassified sample when our k = 1
but in the other hand when our k = 3, we get the most misclassified samples

To sum up, the best prediction is any fold with nearest k = 5 with feature normalization
"""